In [1]:
from datetime import datetime
import pandas as pd

starttime = datetime.now()

### 0. 데이터 불러오기
selloutDataS = pd.read_csv\
        ("../dataset/kopo_channel_result_small.csv",
           encoding="ms949")

In [2]:
selloutDataS

,REGIONID,REGIONNAME,AP1ID,AP1NAME,ACCOUNTID,ACCOUNTNAME,SALESID,SALESNAME,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,TARGET,IDX
0,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02950,201728,2017,28,0,0.0,SALESID0017_ITEM02950
1,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM01161,201728,2017,28,0,0.0,SALESID0017_ITEM01161
2,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02951,201728,2017,28,0,0.0,SALESID0017_ITEM02951
3,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM04036,201728,2017,28,0,0.0,SALESID0017_ITEM04036
4,A01,KOPO,401737,EAST,SALESID0017,서울은행17지점,SALESID0017,서울은행17지점,PG4,PRODUCT12,ITEM02952,201728,2017,28,15,9.2,SALESID0017_ITEM02952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697991,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM00062,201740,2017,40,0,8.3,SALESID0035_ITEM00062
697992,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM04049,201740,2017,40,-3,52.5,SALESID0035_ITEM04049
697993,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM00011,201740,2017,40,0,0.0,SALESID0035_ITEM00011
697994,A01,KOPO,401737,EAST,SALESID0035,서울은행35지점,SALESID0035,서울은행35지점,PG4,PRODUCT12,ITEM03484,201740,2017,40,0,0.0,SALESID0035_ITEM03484


### 1.Pandas 활용

In [1]:
from datetime import datetime
import pandas as pd

starttime = datetime.now()

### 0. 데이터 불러오기
selloutDataS = pd.read_csv\
        ("../dataset/kopo_channel_result_large.csv",
           encoding="ms949")
print("data shape is {}".format(selloutDataS.shape))


### 1. 데이터 전처리: 반품수량 0 처리
def minusToZero(inValue):
    if inValue < 0:
        inValue = 0
    return inValue

data shape is (10469940, 17)


In [4]:
selloutDataS["QTY"] = selloutDataS.QTY.apply(minusToZero)

### 2. 데이터 전처리: 정렬 ["REGIONID","IDX","YEARWEEK"]
sortKey = ["REGIONID","IDX","YEARWEEK"]
selloutDataS.sort_values(sortKey, inplace=True)

### 3. 그룹별 이동평균
groupKey = ["REGIONID","IDX"]

def maFunc(eachgroup):
    eachgroup["MA"] = eachgroup.QTY.\
                        rolling(window=5, 
                                min_periods=1, 
                                center=True).mean()
    return eachgroup[["REGIONID","IDX","YEARWEEK","QTY","MA"]]

finalResult = selloutDataS.groupby(groupKey).apply(maFunc)
finalResult.to_csv("./result_large.csv", 
                   index=False, encoding="ms949")
print("completed")
elapsedtime = datetime.now() - starttime
print( 'time elapsed (hh:mm:ss.ms {})'.\
      format(elapsedtime))

data shape is (10469940, 17)
completed
time elapsed (hh:mm:ss.ms 0:11:51.194153)


In [5]:
finalResult

,REGIONID,IDX,YEARWEEK,QTY,MA
30634,A01,SALESID0001_ITEM00025,201716,28,34.666667
60201,A01,SALESID0001_ITEM00025,201717,39,39.000000
82280,A01,SALESID0001_ITEM00025,201718,37,40.000000
111435,A01,SALESID0001_ITEM00025,201719,52,42.400000
135375,A01,SALESID0001_ITEM00025,201720,44,42.200000
...,...,...,...,...,...
10326196,A15,SALESID0125_ITEM04109,201737,0,0.000000
10358797,A15,SALESID0125_ITEM04109,201738,0,0.000000
10377418,A15,SALESID0125_ITEM04109,201739,0,0.000000
10460447,A15,SALESID0125_ITEM04109,201740,0,0.000000


### 2. Dask 변환 코드

In [101]:
# import pandas as pd
# from dask import delayed

# @delayed
# def mov_average(x):
#     x['mov_avg'] = x.groupby("cust_id")["sales"].apply(
#                             lambda x: x.rolling(window=3, min_periods=1, center=True).mean())
#     return x


# df = pd.DataFrame({'cust_id':['a', 'a', 'a', 'b', 'b'],
#                    'sales': [100, 200, 300, 400, 500]})

# df_1 = mov_average(df).compute()

In [15]:
import warnings

In [16]:
warnings.filterwarnings(action="ignore")

In [17]:
pwd

'C:\\Users\\kopo\\devcode\\Session22 - Data Analytics'

In [18]:
from datetime import datetime
import dask.dataframe as dd
import pandas as pd
from dask import delayed

In [19]:
starttime = datetime.now()

selloutDataS = dd.read_csv\
    ("../dataset/kopo_channel_result_small.csv",
           encoding="ms949")

### 1. 데이터 전처리: 반품수량 0 처리
def minusToZero(inValue):
    if inValue < 0:
        inValue = 0
    return inValue

selloutDataS["QTY"] = \
    selloutDataS.QTY.apply(minusToZero)

selloutDataS["KEY"] = selloutDataS.REGIONID+"_"+selloutDataS.IDX+"_"+selloutDataS.YEARWEEK.astype(str)

### 2. 데이터 전처리: 정렬 ["REGIONID","IDX","YEARWEEK"]
sortKey = ["KEY"]
sortedData = selloutDataS.set_index(sortKey).reset_index(drop=True)

In [20]:
@delayed
def mov_average(x):
    x['mov_avg'] = x.groupby(groupKey)["QTY"].apply(
                            lambda x: x.rolling(window=3, min_periods=1, center=True).mean())
    return x

groupKey = ["REGIONID","IDX"]

# selloutDataS["gkey"] = selloutDataS.REGIONID+"_"+selloutDataS.IDX

finalResult = mov_average(sortedData).compute()

# finalResult = dd.from_pandas(df3, npartitions=2)
finalResult.to_csv("./result_large3.csv", 
                   index=False, encoding="ms949")
print("completed")
elapsedtime = datetime.now() - starttime
print( 'time elapsed (hh:mm:ss.ms {})'.\
      format(elapsedtime))

completed
time elapsed (hh:mm:ss.ms 0:00:46.404278)
